In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl import DGLGraph
import dgl.function as fn
from functools import partial

class RGCNLayer(nn.Module):
    def __init__(self, in_feat, out_feat, num_rels, num_bases=-1, bias=None,
                 activation=None, is_input_layer=False):
        super(RGCNLayer, self).__init__()
        self.in_feat = in_feat
        self.out_feat = out_feat
        self.num_rels = num_rels
        self.num_bases = num_bases
        self.bias = bias
        self.activation = activation
        self.is_input_layer = is_input_layer

        # sanity check
        if self.num_bases <= 0 or self.num_bases > self.num_rels:
            self.num_bases = self.num_rels

        # weight bases in equation (3)
        self.weight = nn.Parameter(torch.Tensor(self.num_bases, self.in_feat,
                                                self.out_feat))
        if self.num_bases < self.num_rels:
            # linear combination coefficients in equation (3)
            self.w_comp = nn.Parameter(torch.Tensor(self.num_rels, self.num_bases))

        # add bias
        if self.bias:
            self.bias = nn.Parameter(torch.Tensor(out_feat))

        # init trainable parameters
        nn.init.xavier_uniform_(self.weight,
                                gain=nn.init.calculate_gain('relu'))
        if self.num_bases < self.num_rels:
            nn.init.xavier_uniform_(self.w_comp,
                                    gain=nn.init.calculate_gain('relu'))
        if self.bias:
            nn.init.xavier_uniform_(self.bias,
                                    gain=nn.init.calculate_gain('relu'))

    def forward(self, g):
        if self.num_bases < self.num_rels:
            # generate all weights from bases (equation (3))
            weight = self.weight.view(self.in_feat, self.num_bases, self.out_feat)
            weight = torch.matmul(self.w_comp, weight).view(self.num_rels,
                                                        self.in_feat, self.out_feat)
        else:
            weight = self.weight

        if self.is_input_layer:
            def message_func(edges):
                # for input layer, matrix multiply can be converted to be
                # an embedding lookup using source node id
                # embed = weight.view(-1, self.out_feat)
                # index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                # index = edges.data['rel_type'] * self.in_feat + edges.src['id']
                # return {'msg': embed[index] * edges.data['norm']}
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['init_h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}                
        else:
            def message_func(edges):
                w = weight[edges.data['rel_type']]
                msg = torch.bmm(edges.src['h'].unsqueeze(1), w).squeeze()
                msg = msg * edges.data['norm']
                return {'msg': msg}

        def apply_func(nodes):
            h = nodes.data['h']
            if self.bias:
                h = h + self.bias
            if self.activation:
                h = self.activation(h)
            return {'h': h}

        g.update_all(message_func, fn.sum(msg='msg', out='h'), apply_func)

Define full R-GCN model
~~~~~~~~~~~~~~~~~~~~~~~



In [ ]:
class Model(nn.Module):
    def __init__(self, num_nodes, h_dim, out_dim, num_rels,node_feature_array,
                 num_bases=-1, num_hidden_layers=1):
        super(Model, self).__init__()
        self.num_nodes = num_nodes
        self.h_dim = h_dim
        self.out_dim = out_dim
        self.num_rels = num_rels
        self.node_feature_array = node_feature_array
        self.num_bases = num_bases
        self.num_hidden_layers = num_hidden_layers
        

        # create rgcn layers
        self.build_model()

        # create initial features
        self.features = self.create_features()

    def build_model(self):
        self.layers = nn.ModuleList()
        # input to hidden
        i2h = self.build_input_layer()
        self.layers.append(i2h)
        # hidden to hidden
        for idx in range(self.num_hidden_layers - 1):
            h2h = self.build_hidden_layer(idx)
            self.layers.append(h2h)
        # hidden to output
        h2o = self.build_output_layer()
        self.layers.append(h2o)

    # initialize feature for each node
    def create_features(self):
        # features = torch.arange(self.num_nodes)
        features = torch.from_numpy(self.node_feature_array)
        return features

    def build_input_layer(self):
        return RGCNLayer(self.node_feature_array.shape[1], self.h_dim[0], self.num_rels, self.num_bases,
                         activation=F.relu, is_input_layer=True)

    def build_hidden_layer(self,idx):
        return RGCNLayer(self.h_dim[idx], self.h_dim[idx+1], self.num_rels, self.num_bases,
                         activation=F.relu)

    def build_output_layer(self):
        return RGCNLayer(self.h_dim[-1], self.out_dim, self.num_rels, self.num_bases,
                         activation=partial(F.softmax, dim=1))

    def forward(self, g):
        if self.features is not None:
            # g.ndata['id'] = self.features
            g.ndata['init_h'] = self.features
        for layer in self.layers:
            layer(g)
        return g.ndata.pop('h')

Handle dataset
~~~~~~~~~~~~~~~~
In this tutorial, we use AIFB dataset from R-GCN paper:



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [ ]:
def get_dist(df,col):
    df_cnt = df.groupby([col]+['etype'])['time'].count().unstack(1,fill_value=0)
    df_dist = pd.DataFrame(df_cnt.values / df_cnt.sum(1).values.reshape(-1,1),
                                               columns=df_cnt.columns,
                                               index=df_cnt.index)
    return df_dist

In [ ]:
from sklearn.preprocessing import LabelEncoder

otc_network = pd.read_csv('raw_data/otc/otc_network.csv',header=None)
otc_network.columns = ['src_raw','dst_raw','etype','time']
etype_encoder = LabelEncoder()
otc_network['etype'] = etype_encoder.fit_transform(otc_network.etype)
otc_gt = pd.read_csv('raw_data/otc/otc_gt.csv',header=None)
otc_gt.columns = ['node_id_raw','label']
otc_gt = otc_gt.drop_duplicates('node_id_raw')

# edge_normの計算
otc_src_cnt = otc_network.groupby(['src_raw','etype'])['time'].count().unstack(1,fill_value=0)

otc_src_dist = pd.DataFrame(otc_src_cnt.values/otc_src_cnt.sum(1).values.reshape(-1,1),
                                                        index=otc_src_cnt.index,
                                                        columns=otc_src_cnt.columns)

merged_network = pd.merge(otc_network,otc_src_dist.stack().reset_index(),on=['src_raw','etype'])


label_encoder = LabelEncoder()
label_encoder.fit(np.hstack((merged_network.src_raw,
                                                   merged_network.dst_raw,
                                                   otc_gt.node_id_raw)))

merged_network['src'] = label_encoder.transform(merged_network.src_raw)

merged_network['dst'] = label_encoder.transform(merged_network.dst_raw)

otc_gt['node_id'] = label_encoder.transform(otc_gt.node_id_raw)
otc_gt['label'] = otc_gt['label'].map(lambda x:1 if x==-1 else 0)

# padding
otc_gt_padded = pd.merge(pd.DataFrame(np.arange(label_encoder.classes_.shape[0])),otc_gt,
                                      left_index=True,right_on='node_id',how='left').fillna(0.5).sort_values('node_id')

num_nodes = label_encoder.classes_.shape[0]
num_rels = merged_network.etype.unique().shape[0]
num_classes = otc_gt.label.unique().shape[0]
labels = otc_gt_padded['label'].values.astype(int).reshape(-1,1)
all_idx = otc_gt['node_id'].values

# edge type and normalization factor
edge_type = torch.from_numpy(merged_network['etype'].values)
edge_norm = torch.from_numpy(merged_network[0].values.astype('float32')).unsqueeze(1)

labels = torch.from_numpy(labels).view(-1)

node_feature_df = pd.concat([get_dist(merged_network,'src'),get_dist(merged_network,'dst')],1).fillna(0).sort_index()
node_feature_array = node_feature_df.values.astype('float32')

known_labels = otc_gt['label'].values

Create graph and model
~~~~~~~~~~~~~~~~~~~~~~~



In [ ]:
# configurations
n_hidden = [32,16] # number of hidden units
n_bases = -1 # -1 # use number of relations as number of bases
n_hidden_layers = 2 # use 1 input layer, 1 output layer, no hidden layer
n_epochs = 300 # epochs to train
lr = 0.01 # learning rate
l2norm = 0.0001 # L2 norm coefficient

## cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
kf = StratifiedKFold(n_splits=10)

In [ ]:
auc_scores = []

for i, (for_train_val_idx, for_test_idx) in enumerate(kf.split(np.arange(len(all_idx)),y=known_labels)):
    train_val_idx = all_idx[for_train_val_idx]
    train_idx, val_idx = train_test_split(train_val_idx,test_size=0.33,stratify=known_labels[for_train_val_idx])
    test_idx = all_idx[for_test_idx]
    # create graph
    g = DGLGraph()
    g.add_nodes(num_nodes)
    g.add_edges(merged_network['src'].values, merged_network['dst'].values)
    g.edata.update({'rel_type': edge_type, 'norm': edge_norm})
    # create model
    model = Model(len(g),
                  n_hidden,
                  num_classes,
                  num_rels,
                  node_feature_array,
                  num_bases=n_bases,
                  num_hidden_layers=n_hidden_layers)
    # optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)
    print("Start {}-th fold".format(i))
    print("==== Train Phase ====")
    model.train()
    best_auc = 0.0
    best_auc_logits = None
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        logits = model.forward(g)
        loss = F.cross_entropy(logits[train_idx], labels[train_idx])
        loss.backward()

        optimizer.step()

        train_auc = roc_auc_score(y_true=labels[train_idx].detach().numpy(),y_score=logits[train_idx].detach().numpy()[:,1])
        train_loss = F.cross_entropy(logits[train_idx], labels[train_idx])
        val_auc = roc_auc_score(y_true=labels[val_idx].detach().numpy(),y_score=logits[val_idx].detach().numpy()[:,1])
        val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
        
        if val_auc >= best_auc:
            best_auc = val_auc
            best_auc_logits = logits
            
        print("Epoch {:05d} | ".format(epoch) +
              "Train AUC: {:.4f} | Train Loss: {:.4f} | ".format(
                  train_auc, loss.item()) +
              "Validation AUC: {:.4f} | Validation loss: {:.4f}".format(
                  val_auc, val_loss.item()))
    print("==== Test Phase ====")
    model.eval()
    test_auc = roc_auc_score(y_true=labels[test_idx].detach().numpy(),y_score=best_auc_logits[test_idx].detach().numpy()[:,1])
    auc_scores.append(test_auc)
    print("test auc : {}".format(test_auc))
    print("=================")

In [ ]:
np.mean(auc_scores)

結果

- (32,16), カーネルは独立
    - amazon :
    - alpha : 0.926 (150 epochs)
    - otc : 0.9601037851 (300 epochs)

# single setting

In [ ]:
# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2norm)

print("start training...")
model.train()
for epoch in range(n_epochs):
    optimizer.zero_grad()
    logits = model.forward(g)
    loss = F.cross_entropy(logits[train_idx], labels[train_idx])
    loss.backward()

    optimizer.step()

    train_acc = torch.sum(logits[train_idx].argmax(dim=1) == labels[train_idx])
    train_acc = train_acc.item() / len(train_idx)
    val_loss = F.cross_entropy(logits[val_idx], labels[val_idx])
    val_acc = torch.sum(logits[val_idx].argmax(dim=1) == labels[val_idx])
    val_acc = val_acc.item() / len(val_idx)
    print("Epoch {:05d} | ".format(epoch) +
          "Train Accuracy: {:.4f} | Train Loss: {:.4f} | ".format(
              train_acc, loss.item()) +
          "Validation Accuracy: {:.4f} | Validation loss: {:.4f}".format(
              val_acc, val_loss.item()))

In [ ]:
y_pred = logits[val_idx].detach().numpy()[:,1]

y_true = labels[val_idx].detach().numpy()

In [ ]:
roc_auc_score(y_true=labels[val_idx].detach().numpy(),y_score=logits[val_idx].detach().numpy()[:,1])

In [ ]:
_ = plt.hist(logits[val_idx].detach().numpy()[:,1][y_true==1],alpha=0.5)
_ = plt.hist(logits[val_idx].detach().numpy()[:,1][y_true==0],alpha=0.5)